# Azure AI Search और NVIDIA NIM और LlamaIndex का एकीकरण

इस नोटबुक में, हम NVIDIA के AI मॉडल और LlamaIndex का उपयोग करके एक शक्तिशाली Retrieval-Augmented Generation (RAG) पाइपलाइन बनाने का प्रदर्शन करेंगे। हम NVIDIA के LLMs और embeddings का उपयोग करेंगे, उन्हें Azure AI Search के साथ एकीकृत करेंगे और RAG का उपयोग करके खोज की गुणवत्ता और दक्षता को बढ़ाएंगे।

## लाभ
- **स्केलेबिलिटी**: NVIDIA के बड़े भाषा मॉडल और Azure AI Search का उपयोग करके स्केलेबल और प्रभावी रिट्रीवल प्राप्त करें।
- **लागत प्रभावशीलता**: कुशल वेक्टर स्टोरेज और हाइब्रिड सर्च तकनीकों के साथ खोज और रिट्रीवल को अनुकूलित करें।
- **उच्च प्रदर्शन**: शक्तिशाली LLMs को वेक्टराइज्ड सर्च के साथ जोड़कर तेज़ और सटीक उत्तर प्राप्त करें।
- **गुणवत्ता**: प्रासंगिक दस्तावेज़ों के साथ LLM प्रतिक्रियाओं को आधार बनाकर उच्च खोज गुणवत्ता बनाए रखें।

## आवश्यकताएँ
- 🐍 Python 3.9 या उससे अधिक
- 🔗 [Azure AI Search Service](https://learn.microsoft.com/azure/search/)
- 🔗 NVIDIA API Key, NVIDIA के LLMs और Embeddings तक पहुँच के लिए NVIDIA NIM माइक्रोसर्विसेज के माध्यम से

## शामिल विशेषताएँ
- ✅ NVIDIA LLM एकीकरण (हम [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe) का उपयोग करेंगे)
- ✅ NVIDIA Embeddings (हम [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5) का उपयोग करेंगे)
- ✅ Azure AI Search उन्नत रिट्रीवल मोड्स
- ✅ LlamaIndex के साथ दस्तावेज़ इंडेक्सिंग
- ✅ Azure AI Search और LlamaIndex के साथ NVIDIA LLMs का उपयोग करके RAG

चलो शुरू करते हैं!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## स्थापना और आवश्यकताएँ
Python संस्करण >3.10 का उपयोग करके एक Python वातावरण बनाएं।

## शुरुआत करें!


NVIDIA AI Foundation मॉडल का उपयोग करने के लिए आपको `NVIDIA_API_KEY` की आवश्यकता होगी:
1) [NVIDIA](https://build.nvidia.com/explore/discover) के साथ एक मुफ्त खाता बनाएं।
2) अपनी पसंद के मॉडल पर क्लिक करें।
3) Input के तहत, Python टैब चुनें, और **Get API Key** पर क्लिक करें, फिर **Generate Key** पर क्लिक करें।
4) उत्पन्न कुंजी को कॉपी करें और इसे NVIDIA_API_KEY के रूप में सहेजें। वहां से, आपको endpoints तक पहुंच प्राप्त हो जाएगी।


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## LLM और एम्बेडिंग का उपयोग करके RAG उदाहरण
### 1) LLM को प्रारंभ करें
`llama-index-llms-nvidia`, जिसे NVIDIA का LLM कनेक्टर भी कहा जाता है, आपको NVIDIA API कैटलॉग में उपलब्ध संगत मॉडलों से जुड़ने और उत्पन्न करने की अनुमति देता है। चैट पूर्णता मॉडलों की सूची के लिए यहां देखें: https://build.nvidia.com/search?term=Text-to-Text

यहां हम **mixtral-8x7b-instruct-v0.1** का उपयोग करेंगे।


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) एम्बेडिंग को प्रारंभ करें  
`llama-index-embeddings-nvidia`, जिसे NVIDIA का Embeddings कनेक्टर भी कहा जाता है, आपको NVIDIA API कैटलॉग में उपलब्ध संगत मॉडलों से जुड़ने और उत्पन्न करने की अनुमति देता है। हमने `nvidia/nv-embedqa-e5-v5` को एम्बेडिंग मॉडल के रूप में चुना। टेक्स्ट एम्बेडिंग मॉडलों की सूची के लिए यहां देखें: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) अपने डेटा पर प्रश्न पूछने के लिए एक क्वेरी इंजन बनाएं

यहाँ Azure AI Search में शुद्ध वेक्टर खोज का उपयोग करके और हमारे LLM (Phi-3.5-MOE) के उत्तर को आधार बनाते हुए एक क्वेरी दी गई है।


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

यहाँ Azure AI Search में हाइब्रिड सर्च का उपयोग करते हुए एक क्वेरी है।


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### वेक्टर खोज विश्लेषण
LLM प्रतिक्रिया ने स्रोत पाठ में उल्लिखित रूस के शेयर बाजार के प्रमुख आर्थिक परिणामों को सटीक रूप से कैप्चर किया है। विशेष रूप से, यह बताता है कि रूसी शेयर बाजार में भारी गिरावट आई, जिससे इसका मूल्य 40% कम हो गया, और चल रही स्थिति के कारण ट्रेडिंग को निलंबित कर दिया गया। यह प्रतिक्रिया स्रोत में दी गई जानकारी के साथ अच्छी तरह मेल खाती है, यह दर्शाते हुए कि LLM ने रूस की कार्रवाइयों और लगाए गए प्रतिबंधों के परिणामस्वरूप शेयर बाजार पर प्रभाव से संबंधित प्रासंगिक विवरणों को सही ढंग से पहचाना और सारांशित किया।

#### स्रोत नोड्स पर टिप्पणी
स्रोत नोड्स ने अंतरराष्ट्रीय प्रतिबंधों के कारण रूस को झेलने वाले आर्थिक परिणामों का विस्तृत विवरण प्रदान किया है। पाठ में यह बताया गया है कि रूसी शेयर बाजार ने अपना 40% मूल्य खो दिया और ट्रेडिंग को निलंबित कर दिया गया। इसके अलावा, यह अन्य आर्थिक प्रभावों का भी उल्लेख करता है, जैसे रूबल का अवमूल्यन और रूस की अर्थव्यवस्था का व्यापक अलगाव। LLM प्रतिक्रिया ने इन नोड्स से महत्वपूर्ण बिंदुओं को प्रभावी ढंग से संक्षेपित किया, विशेष रूप से शेयर बाजार के प्रभाव पर ध्यान केंद्रित करते हुए, जैसा कि क्वेरी में अनुरोध किया गया था।


अब, आइए एक प्रश्न पर नज़र डालते हैं जहाँ हाइब्रिड सर्च एक ठोस उत्तर प्रदान नहीं करता:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### हाइब्रिड सर्च: LLM प्रतिक्रिया विश्लेषण
हाइब्रिड सर्च उदाहरण में LLM की प्रतिक्रिया यह संकेत देती है कि दिए गए संदर्भ में रूस के यूक्रेन पर आक्रमण की सटीक तारीख का उल्लेख नहीं है। यह प्रतिक्रिया दर्शाती है कि LLM स्रोत दस्तावेजों में उपलब्ध जानकारी का उपयोग कर रहा है, लेकिन यह भी स्वीकार करता है कि पाठ में सटीक विवरण अनुपस्थित है।

प्रतिक्रिया सही तरीके से पहचानती है कि संदर्भ में रूस की आक्रामकता से संबंधित घटनाओं का उल्लेख है, लेकिन आक्रमण की विशिष्ट तारीख को स्पष्ट नहीं किया गया है। यह LLM की क्षमता को दर्शाता है कि वह प्रदान की गई जानकारी को समझ सकता है और सामग्री में मौजूद कमियों को पहचान सकता है। LLM प्रभावी रूप से उपयोगकर्ता को सटीक तारीख के लिए बाहरी स्रोतों या ऐतिहासिक रिकॉर्ड की तलाश करने के लिए प्रेरित करता है, और यह तब सतर्कता दिखाता है जब जानकारी अधूरी होती है।

### स्रोत नोड्स विश्लेषण
हाइब्रिड सर्च उदाहरण में स्रोत नोड्स में एक भाषण के अंश शामिल हैं, जो यूक्रेन में रूस की कार्रवाइयों के प्रति अमेरिका की प्रतिक्रिया पर चर्चा करते हैं। ये नोड्स व्यापक भू-राजनीतिक प्रभाव और आक्रमण के जवाब में अमेरिका और उसके सहयोगियों द्वारा उठाए गए कदमों पर जोर देते हैं, लेकिन वे विशिष्ट आक्रमण तारीख का उल्लेख नहीं करते। यह LLM प्रतिक्रिया के साथ मेल खाता है, जो सही तरीके से पहचानता है कि संदर्भ में सटीक तारीख की जानकारी अनुपस्थित है।


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### हाइब्रिड w/Reranking: LLM प्रतिक्रिया विश्लेषण
हाइब्रिड w/Reranking उदाहरण में, LLM प्रतिक्रिया अतिरिक्त संदर्भ प्रदान करती है, यह बताते हुए कि घटना भाषण दिए जाने से छह दिन पहले हुई थी। यह संकेत करता है कि LLM भाषण के समय के आधार पर आक्रमण की तारीख का अनुमान लगा सकता है, हालांकि इसे सटीकता के लिए भाषण की सही तारीख जानने की आवश्यकता होती है।

यह प्रतिक्रिया संदर्भ संकेतों का उपयोग करके अधिक जानकारीपूर्ण उत्तर देने की बेहतर क्षमता को प्रदर्शित करती है। यह रीरैंकिंग के लाभ को उजागर करती है, जहां LLM अधिक प्रासंगिक जानकारी तक पहुंच सकता है और उसे प्राथमिकता दे सकता है ताकि वांछित विवरण (जैसे, आक्रमण की तारीख) के करीब अनुमान प्रदान किया जा सके।

### स्रोत नोड्स विश्लेषण
इस उदाहरण में स्रोत नोड्स में रूस के आक्रमण के समय का संदर्भ शामिल है, विशेष रूप से यह उल्लेख करते हुए कि यह भाषण से छह दिन पहले हुआ था। हालांकि सटीक तारीख अभी भी स्पष्ट रूप से नहीं बताई गई है, नोड्स अस्थायी संदर्भ प्रदान करते हैं जो LLM को अधिक सूक्ष्म प्रतिक्रिया देने में सक्षम बनाते हैं। इस विवरण का समावेश यह दिखाता है कि रीरैंकिंग कैसे LLM की क्षमता को सुधार सकता है ताकि प्रदान किए गए संदर्भ से जानकारी निकालने और अनुमान लगाने में मदद मिले, जिससे अधिक सटीक और जानकारीपूर्ण प्रतिक्रिया प्राप्त हो सके।


**नोट:**
इस नोटबुक में, हमने NVIDIA API कैटलॉग से NVIDIA NIM माइक्रोसर्विसेस का उपयोग किया है।  
उपरोक्त APIs, `NVIDIA (llms)`, `NVIDIAEmbedding`, और [Azure AI Search Semantic Hybrid Retrieval (बिल्ट-इन रीरैंकिंग)](https://learn.microsoft.com/azure/search/semantic-search-overview)। ध्यान दें कि उपरोक्त APIs सेल्फ-होस्टेड माइक्रोसर्विसेस को भी सपोर्ट कर सकते हैं।  

**उदाहरण:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम उत्तरदायी नहीं हैं।
